In [5]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

# Setup

In [7]:
# Get source file
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()
print('corpus length:', len(text))

corpus length: 600901


In [9]:
# Create a vocabulary of chars in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 86


In [10]:
chars.insert(0, "\0")

In [11]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz'

In [14]:
char_indices = dict((c,i) for i,c in enumerate(chars))
indices_char = dict(enumerate(chars))

In [24]:
# convert whole text into (char)indices, a new representation
idx = [char_indices[c] for c in text]

In [25]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [26]:
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

In [27]:
cs=3
c1_dat = [idx[i]   for i in xrange(0, len(idx)-1-cs, cs)]
c2_dat = [idx[i+1] for i in xrange(0, len(idx)-1-cs, cs)]
c3_dat = [idx[i+2] for i in xrange(0, len(idx)-1-cs, cs)]
c4_dat = [idx[i+3] for i in xrange(0, len(idx)-1-cs, cs)]

In [28]:
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

In [29]:
y = np.stack(c4_dat[:-2])

In [30]:
x1[:4], x2[:4], x3[:4]

(array([40, 30, 29,  1]), array([42, 25,  1, 43]), array([29, 27,  1, 45]))

In [31]:
y[:3]

array([30, 29,  1])

In [32]:
x1.shape, y.shape

((200297,), (200297,))

In [33]:
n_fac = 42

In [34]:
def embedding_input(name, n_in, n_out):
    inp = Input (shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [36]:
c1_in, c1 = embedding_input('c1', vocab_size, n_fac)
c2_in, c2 = embedding_input('c2', vocab_size, n_fac)
c3_in, c3 = embedding_input('c3', vocab_size, n_fac)

In [39]:
?Embedding